## installing required libraries

In [3]:
!pip install bing-image-downloader gradio torch torchvision


## Download Real & Fake Brand Logos

In [5]:
import os
from bing_image_downloader import downloader

def download_images():
    categories = {
        "real": ["Nike logo", "Adidas logo", "Puma logo", "Reebok logo"],
        "fake": ["Fake Nike logo", "Fake Adidas logo", "Fake Puma logo", "Fake Reebok logo"]
    }

    dataset_path = "dataset_main"
    os.makedirs(dataset_path, exist_ok=True)

    for label, queries in categories.items():
        label_dir = os.path.join(dataset_path, label)
        os.makedirs(label_dir, exist_ok=True)

        for query in queries:
            downloader.download(query, limit=30, output_dir=label_dir, adult_filter_off=True, force_replace=False)


download_images()


[%] Downloading Images to /content/dataset_main/real/Nike logo


[!!]Indexing page: 1

[%] Indexed 28 Images on Page 1.


[%] Downloading Image #1 from https://logos-world.net/wp-content/uploads/2020/04/Nike-Logo-1978-present.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://logolook.net/wp-content/uploads/2021/06/Nike-Logo-History.png
[%] File Downloaded !

[%] Downloading Image #3 from https://logo-marque.com/wp-content/uploads/2020/05/Nike-Logo-Histoire.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://cdn.britannica.com/50/213250-050-02322AA8/Nike-logo.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://www.pixelstalk.net/wp-content/uploads/2015/12/nike-logo-wallpaper1.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http://www.pixelstalk.net/wp-content/uploads/2015/12/nike-logo-wallpapers-white-black.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://logos-download.com/wp-content/uploads/2016/03/Nike_logo_ball.png
[%]

## Training Fake Brand Logo Detector Using ResNet

In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models, datasets
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset_path = "dataset_main"
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Load Pretrained ResNet model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))  # Adjust output

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to("cpu"), labels.to("cpu")  # Move to CPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save trained model
torch.save(model.state_dict(), "brand_detector.pth")
print("Model Training Complete!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 155MB/s]
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/5], Loss: 0.3521
Epoch [2/5], Loss: 0.3228
Epoch [3/5], Loss: 0.0424
Epoch [4/5], Loss: 0.0693
Epoch [5/5], Loss: 0.1856
Model Training Complete!


## Gradio Interface

In [7]:
import gradio as gr
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Load trained model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("brand_detector.pth", map_location=torch.device("cpu")))
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Label mapping
class_labels = ["Real Brand Logo", "Fake Brand Logo"]

# Function to predict image
def predict(img):
    img = transform(img).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output, 1)
    return class_labels[predicted.item()]

# Create Gradio Interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Fake Brand Logo Detector",
    description="Upload a brand logo to check if it's real or fake."
)

interface.launch()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-7-0af146aeeca9>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detail

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d53b0f88517d2db2d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
